In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
register_matplotlib_converters()
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sms
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/STECH/time-series

/content/drive/MyDrive/STECH/time-series


In [4]:
df = pd.read_csv('FX_EURUSD, 1D.csv')

In [5]:
cols = ['time','close']
df = df[cols]

In [6]:
df = df.astype({'time':'datetime64'})

In [7]:
df['date'] = pd.to_datetime(df['time'], unit='s').dt.date
df['date'] = pd.to_datetime(df['date'])
df.drop('time',axis=1,inplace=True)

In [8]:
df['close_diff_1'] = df['close'] - df['close'].shift(1)
df = df[1:]
df.head()

,close,date,close_diff_1
1,1.29538,2012-10-11,0.00270
2,1.29490,2012-10-14,-0.00048
3,1.30529,2012-10-15,0.01039
4,1.31188,2012-10-16,0.00659
5,1.30664,2012-10-17,-0.00524


In [9]:
test_size = 0.2
train_samples = int((1-test_size)*len(df))
train_samples

1994

In [10]:
train_data = df[:train_samples]
test_data = df[train_samples:]

In [11]:
train_data

,close,date,close_diff_1
1,1.29538,2012-10-11,0.00270
2,1.29490,2012-10-14,-0.00048
3,1.30529,2012-10-15,0.01039
4,1.31188,2012-10-16,0.00659
5,1.30664,2012-10-17,-0.00524
...,...,...,...
1990,1.12057,2020-06-17,-0.00373
1991,1.11733,2020-06-18,-0.00324
1992,1.12601,2020-06-21,0.00868
1993,1.13073,2020-06-22,0.00472


In [12]:
test_data

,close,date,close_diff_1
1995,1.12174,2020-06-24,-0.00328
1996,1.12162,2020-06-25,-0.00012
1997,1.12412,2020-06-28,0.00250
1998,1.12331,2020-06-29,-0.00081
1999,1.12506,2020-06-30,0.00175
...,...,...,...
2489,1.05600,2022-05-19,-0.00217
2490,1.06885,2022-05-22,0.01285
2491,1.07314,2022-05-23,0.00429
2492,1.06763,2022-05-24,-0.00551


In [13]:
train_data.Timestamp = pd.to_datetime(train_data.date, format='%Y-%m-%d')
train_data.index = train_data.Timestamp
train_data = train_data.resample('D').sum()

test_data.Timestamp = pd.to_datetime(test_data.date, format='%Y-%m-%d')
test_data.index = test_data.Timestamp
test_data = test_data.resample('D').sum()

In [14]:
pred_start_date = test_data.index[0]
pred_end_date=test_data.index[-1]

In [15]:
from statsmodels.tsa.arima_model import ARMA

In [16]:
model_ARMA = ARMA(train_data['close'], order=(3,0))
model_ARMA_fit = model_ARMA.fit()
model_ARMA_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                  close   No. Observations:                 2813
Model:                     ARMA(3, 0)   Log Likelihood               -1726.678
Method:                       css-mle   S.D. of innovations              0.447
Date:                Sat, 28 May 2022   AIC                           3463.356
Time:                        02:35:37   BIC                           3493.066
Sample:                    10-11-2012   HQIC                          3474.078
                         - 06-23-2020                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.8394      0.008    111.373      0.000       0.825       0.854
ar.L1.close     0.4215      0.019     22.395      0.000       0.385       0.458
ar.L2.close    -0.4777      0.018    -26.051      0.000      -0.514      -0.442
ar.L3.close    -0.0622      0.019     -3.304      0.001      -0.099      -0.025
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            0.4974           -1.2672j            1.3613           -0.1905
AR.2            0.4974           +1.2672j            1.3613            0.1905
AR.3           -8.6754           -0.0000j            8.6754           -0.5000
-----------------------------------------------------------------------------
"""

In [17]:
pred_start_date = test_data.index[0]
pred_end_date = test_data.index[-1]

In [20]:
from datetime import datetime, timedelta

In [23]:
for end_date in test_data.index:
  print(end_date-timedelta(days=1))

2020-06-23 00:00:00
2020-06-24 00:00:00
2020-06-25 00:00:00
2020-06-26 00:00:00
2020-06-27 00:00:00
2020-06-28 00:00:00
2020-06-29 00:00:00
2020-06-30 00:00:00
2020-07-01 00:00:00
2020-07-02 00:00:00
2020-07-03 00:00:00
2020-07-04 00:00:00
2020-07-05 00:00:00
2020-07-06 00:00:00
2020-07-07 00:00:00
2020-07-08 00:00:00
2020-07-09 00:00:00
2020-07-10 00:00:00
2020-07-11 00:00:00
2020-07-12 00:00:00
2020-07-13 00:00:00
2020-07-14 00:00:00
2020-07-15 00:00:00
2020-07-16 00:00:00
2020-07-17 00:00:00
2020-07-18 00:00:00
2020-07-19 00:00:00
2020-07-20 00:00:00
2020-07-21 00:00:00
2020-07-22 00:00:00
2020-07-23 00:00:00
2020-07-24 00:00:00
2020-07-25 00:00:00
2020-07-26 00:00:00
2020-07-27 00:00:00
2020-07-28 00:00:00
2020-07-29 00:00:00
2020-07-30 00:00:00
2020-07-31 00:00:00
2020-08-01 00:00:00
2020-08-02 00:00:00
2020-08-03 00:00:00
2020-08-04 00:00:00
2020-08-05 00:00:00
2020-08-06 00:00:00
2020-08-07 00:00:00
2020-08-08 00:00:00
2020-08-09 00:00:00
2020-08-10 00:00:00
2020-08-11 00:00:00


In [21]:
predictions_rolling = pd.Series()
for end_date in test_data.index:
  train_data = df[:end_date - timedelta(days=1)]
  model = ARMA(train_data, order=(3,0))
  model_fit = model.fit()
  pred = model_fit.predict(end_date)
  predictions_rolling.loc[end_date] = pred.loc[end_date]

TypeError: ignored